In [ ]:
import rnn_sweep

In [2]:
import test_rnn_plain

C:\Users\danie\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Experiment: Training size: 100, Batch size: 32, Epochs: 2500, Dropout: 0.0
Num_batch:  32
EncoderRNN(
  (embedding): Embedding(4300, 256)
  (cnn): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (gru): GRU(128, 256)
)
DecoderRNN(
  (embedding): Embedding(16, 256)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=16, bias=True)
  (softmax): LogSoftmax(dim=1)
)
0m 20s (- 3m 2s) (250 10%) 1.7838
0m 41s (- 2m 46s) (500 20%) 1.4963
1m 1s (- 2m 24s) (750 30%) 1.3880
1m 22s (- 2m 3s) (1000 40%) 1.2570
1m 42s (- 1m 42s) (1250 50%) 1.0545
2m 2s (- 1m 21s) (1500 60%) 0.8615
2m 23s (- 1m 1s) (1750 70%) 0.6325
2m 43s (- 0m 40s) (2000 80%) 0.3760
3m 4s (- 0m 20s) (2250 90%) 0.2233
3m 24s (- 0m 0s) (2500 100%) 0.1308
Accuracy score on training set:  1.0
MAE on the test set:  1346.969
